In [1]:
import tensorflow as tf 
import tensorflow_hub as hub
import tensorflow_text

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import string

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_squared_error

from official.nlp import optimization

import matplotlib.pyplot as plt

In [5]:
model_load_1 = tf.keras.models.load_model("saved_model/3",  compile=False)
model_load_2 = tf.keras.models.load_model("saved_model/4",  compile=False)
model_load_3 = tf.keras.models.load_model("saved_model/5",  compile=False)

In [6]:
path =  "E:/LOMBA/Kaggle/Comp/Coommon readilbility/"
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
sample_submission = pd.read_csv(path + 'sample_submission.csv')

STOPWORDS = set(stopwords.words("english"))
def text_cleaning(text):
    text_clean = " ".join([word for word in word_tokenize(text) if not word in STOPWORDS])
    text = re.sub("\s+"," ", text_clean) # remove extra spaces
    text = ''.join([k for k in text if k not in string.punctuation]) # remove punctuation
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text)).strip() # remove unnecessary literals and Perform Case Normalization
    text = ''.join([i for i in text if not i.isdigit()])
    return text


In [7]:
train["clean_txt"]= train.excerpt.apply(text_cleaning)
train.head()

,id,url_legal,license,excerpt,target,standard_error,clean_txt
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,When young people returned ballroom presented ...
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,All dinner time Mrs Fayre somewhat silent eyes...
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,As Roger predicted snow departed quickly came ...
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,And outside palace great garden walled round f...
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,Once upon time Three Bears lived together hous...


In [12]:
pred_1 = model_load_1.predict(train.clean_txt.to_numpy())
pred_2 = model_load_2.predict(train.clean_txt.to_numpy())
pred_3 = model_load_3.predict(train.clean_txt.to_numpy())

In [17]:
pred_1.shape

(2834, 1)

In [21]:
train_model = np.concatenate((pred_1, pred_2, pred_3), axis=1)
target = train.target.to_numpy()

In [23]:
train_x, test_x, train_y, test_y = train_test_split(train_model, target, test_size=0.2, shuffle=True, random_state=42)

In [24]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor

In [25]:
reg1 = GradientBoostingRegressor(random_state=1)
reg2 = RandomForestRegressor(random_state=1)
reg3 = LinearRegression()
ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])


In [26]:
ereg = ereg.fit(train_x, train_y)

In [32]:
y_train_pred = ereg.predict(train_x)
print (np.sqrt(mean_squared_error(y_train_pred, train_y)))

y_test_pred = ereg.predict(test_x)
print (np.sqrt(mean_squared_error(y_test_pred, test_y)))

0.23766396213668914
0.5178163431267645


In [34]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsRegressor
estimators = [('ridge', RidgeCV()),
              ('lasso', LassoCV(random_state=42)),
              ('knr', KNeighborsRegressor(n_neighbors=20,metric='euclidean'))]

In [35]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
final_estimator = GradientBoostingRegressor(
     n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1,
    random_state=42)
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=final_estimator)

In [36]:
reg.fit(train_x, train_y)

StackingRegressor(estimators=[('ridge',
                               RidgeCV(alphas=array([ 0.1,  1. , 10. ]))),
                              ('lasso', LassoCV(random_state=42)),
                              ('knr',
                               KNeighborsRegressor(metric='euclidean',
                                                   n_neighbors=20))],
                  final_estimator=GradientBoostingRegressor(max_features=1,
                                                            min_samples_leaf=25,
                                                            n_estimators=25,
                                                            random_state=42,
                                                            subsample=0.5))

In [37]:
y_train_pred = reg.predict(train_x)
print (np.sqrt(mean_squared_error(y_train_pred, train_y)))

y_test_pred = reg.predict(test_x)
print (np.sqrt(mean_squared_error(y_test_pred, test_y)))

0.3189197567454781
0.5111223903766028


In [39]:
pred = (pred_1 + pred_2 + pred_3)/3
print (np.sqrt(mean_squared_error(pred, target)))

0.3927554874932165


In [55]:
pred = (0.2*pred_1 + 0.15*pred_2 + 0.65*pred_3)
print (np.sqrt(mean_squared_error(pred, target)))

0.3838207590079349


In [57]:
pred = (0.2*pred_1 + 0.13*pred_2 + 0.67*pred_3)
print (np.sqrt(mean_squared_error(pred, target)))

0.3828976587517151


In [58]:
import optuna

In [66]:
def objective(trial):
    param = {
        "n_1": trial.suggest_float("n_1", 0.0, 1.0),
        "n_2": trial.suggest_float("n_2", 0.0, 1.0),
        "n_3": trial.suggest_float("n_3", 0.0, 1.0)
    }

    pred = (param["n_1"]*pred_1 + param["n_2"]*pred_2 + param["n_3"]*pred_3)
    td = (np.sqrt(mean_squared_error(pred, target)))
    return td

In [67]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-06-23 19:14:08,665] A new study created in memory with name: no-name-d1e98791-54a8-43ca-ba5f-020abb433ee8
[I 2021-06-23 19:14:08,669] Trial 0 finished with value: 0.4071949379843488 and parameters: {'n_1': 0.9606194165465876, 'n_2': 0.045187660169070165, 'n_3': 0.09109479637004492}. Best is trial 0 with value: 0.4071949379843488.
[I 2021-06-23 19:14:08,672] Trial 1 finished with value: 0.6061333411379686 and parameters: {'n_1': 0.5632866085104451, 'n_2': 0.016228802278472565, 'n_3': 0.7649346595600709}. Best is trial 0 with value: 0.4071949379843488.
[I 2021-06-23 19:14:08,674] Trial 2 finished with value: 1.3983299205996524 and parameters: {'n_1': 0.927353945636788, 'n_2': 0.8436345235790245, 'n_3': 0.24673926023900294}. Best is trial 0 with value: 0.4071949379843488.
[I 2021-06-23 19:14:08,676] Trial 3 finished with value: 0.5283107730173715 and parameters: {'n_1': 0.028795440397293648, 'n_2': 0.9384446714938186, 'n_3': 0.2577313995376094}. Best is trial 0 with value: 0.40719

[I 2021-06-23 19:14:09,125] Trial 34 finished with value: 0.4893340153800635 and parameters: {'n_1': 0.5486586319424733, 'n_2': 0.0046001665987078525, 'n_3': 0.21816809467635756}. Best is trial 33 with value: 0.3703411352222359.
[I 2021-06-23 19:14:09,137] Trial 35 finished with value: 0.4353913571585158 and parameters: {'n_1': 0.7205535447251632, 'n_2': 0.003905612931263481, 'n_3': 0.44308649298301644}. Best is trial 33 with value: 0.3703411352222359.
[I 2021-06-23 19:14:09,146] Trial 36 finished with value: 0.38886587020594426 and parameters: {'n_1': 0.6177538341959214, 'n_2': 0.04788490552518671, 'n_3': 0.2665857937681686}. Best is trial 33 with value: 0.3703411352222359.
[I 2021-06-23 19:14:09,155] Trial 37 finished with value: 0.8705382613593272 and parameters: {'n_1': 0.530721233909999, 'n_2': 0.7040195042184101, 'n_3': 0.3522427615459787}. Best is trial 33 with value: 0.3703411352222359.
[I 2021-06-23 19:14:09,209] Trial 38 finished with value: 0.3781172913040141 and parameters:

[I 2021-06-23 19:14:09,500] Trial 69 finished with value: 0.3813046398055874 and parameters: {'n_1': 0.5821035927884236, 'n_2': 0.13585027860644483, 'n_3': 0.2684031401328013}. Best is trial 33 with value: 0.3703411352222359.
[I 2021-06-23 19:14:09,510] Trial 70 finished with value: 0.3766954766355193 and parameters: {'n_1': 0.6321053648659533, 'n_2': 0.06081210399726848, 'n_3': 0.3341019057344003}. Best is trial 33 with value: 0.3703411352222359.
[I 2021-06-23 19:14:09,520] Trial 71 finished with value: 0.37696071023194944 and parameters: {'n_1': 0.6341805136996377, 'n_2': 0.05895447186604104, 'n_3': 0.33703781610123573}. Best is trial 33 with value: 0.3703411352222359.
[I 2021-06-23 19:14:09,530] Trial 72 finished with value: 0.37586936377453956 and parameters: {'n_1': 0.6369883276494241, 'n_2': 0.058557489282546914, 'n_3': 0.3167386604483605}. Best is trial 33 with value: 0.3703411352222359.
[I 2021-06-23 19:14:09,540] Trial 73 finished with value: 0.38287555755322406 and parameters

Number of finished trials: 100
Best trial:
  Value: 0.3703411352222359
  Params: 
    n_1: 0.5416157034663671
    n_2: 0.001419402645076625
    n_3: 0.4465225550555642


In [68]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000)
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-06-23 19:14:37,807] A new study created in memory with name: no-name-a6efa6bd-c95f-42f5-b22a-9f9c310b4dd2
[I 2021-06-23 19:14:37,810] Trial 0 finished with value: 0.7382913728603094 and parameters: {'n_1': 0.5392741085827872, 'n_2': 0.5063322796509193, 'n_3': 0.4268392110623701}. Best is trial 0 with value: 0.7382913728603094.
[I 2021-06-23 19:14:37,812] Trial 1 finished with value: 0.7039448987976002 and parameters: {'n_1': 0.8185892004495157, 'n_2': 0.1338650650555624, 'n_3': 0.4905280483303196}. Best is trial 1 with value: 0.7039448987976002.
[I 2021-06-23 19:14:37,814] Trial 2 finished with value: 1.8942642843427175 and parameters: {'n_1': 0.7823245175012284, 'n_2': 0.6529237999308429, 'n_3': 0.9400364145478861}. Best is trial 1 with value: 0.7039448987976002.
[I 2021-06-23 19:14:37,817] Trial 3 finished with value: 0.9183899774002522 and parameters: {'n_1': 0.3820834665789534, 'n_2': 0.7994786531138477, 'n_3': 0.4404029263255973}. Best is trial 1 with value: 0.703944898797

[I 2021-06-23 19:14:38,080] Trial 34 finished with value: 0.4130887897947661 and parameters: {'n_1': 0.29387384564060315, 'n_2': 0.04414744523400008, 'n_3': 0.5264319210206092}. Best is trial 29 with value: 0.37577272942702517.
[I 2021-06-23 19:14:38,090] Trial 35 finished with value: 0.5751086047250961 and parameters: {'n_1': 0.5028991115037905, 'n_2': 0.15372580493424232, 'n_3': 0.6604598405568822}. Best is trial 29 with value: 0.37577272942702517.
[I 2021-06-23 19:14:38,100] Trial 36 finished with value: 0.3804221220256586 and parameters: {'n_1': 0.5565182195098388, 'n_2': 0.003340126357601725, 'n_3': 0.5027338579809004}. Best is trial 29 with value: 0.37577272942702517.
[I 2021-06-23 19:14:38,108] Trial 37 finished with value: 0.6904476007487211 and parameters: {'n_1': 0.6904392820376096, 'n_2': 0.08647689182035935, 'n_3': 0.6492705044979525}. Best is trial 29 with value: 0.37577272942702517.
[I 2021-06-23 19:14:38,117] Trial 38 finished with value: 0.6444563547676664 and parameter

[I 2021-06-23 19:14:38,412] Trial 69 finished with value: 0.47785959383586835 and parameters: {'n_1': 0.4359396629390191, 'n_2': 0.08262300358083045, 'n_3': 0.693060686736595}. Best is trial 29 with value: 0.37577272942702517.
[I 2021-06-23 19:14:38,421] Trial 70 finished with value: 0.5013887096821283 and parameters: {'n_1': 0.5905886725140247, 'n_2': 0.24835946545690163, 'n_3': 0.4067150806824507}. Best is trial 29 with value: 0.37577272942702517.
[I 2021-06-23 19:14:38,431] Trial 71 finished with value: 0.3972216107572992 and parameters: {'n_1': 0.04594555480670723, 'n_2': 0.13445500896346857, 'n_3': 0.7533828846461019}. Best is trial 29 with value: 0.37577272942702517.
[I 2021-06-23 19:14:38,441] Trial 72 finished with value: 0.39138343820312993 and parameters: {'n_1': 0.0806023085928384, 'n_2': 0.14717262004841392, 'n_3': 0.7800181787602272}. Best is trial 29 with value: 0.37577272942702517.
[I 2021-06-23 19:14:38,449] Trial 73 finished with value: 0.45171934616044457 and paramete

[I 2021-06-23 19:14:38,734] Trial 103 finished with value: 0.3892058605656088 and parameters: {'n_1': 0.2436749150490214, 'n_2': 0.032625627060317186, 'n_3': 0.7794128992181936}. Best is trial 29 with value: 0.37577272942702517.
[I 2021-06-23 19:14:38,745] Trial 104 finished with value: 0.6779142133476205 and parameters: {'n_1': 0.21477431634857136, 'n_2': 0.014982730489416, 'n_3': 0.3492826419060396}. Best is trial 29 with value: 0.37577272942702517.
[I 2021-06-23 19:14:38,754] Trial 105 finished with value: 0.4966182268017142 and parameters: {'n_1': 0.2837111469341674, 'n_2': 0.09161762611135818, 'n_3': 0.8466330749777486}. Best is trial 29 with value: 0.37577272942702517.
[I 2021-06-23 19:14:38,765] Trial 106 finished with value: 1.2242944047662083 and parameters: {'n_1': 0.4949977185253482, 'n_2': 0.5817349914110658, 'n_3': 0.7832394206680644}. Best is trial 29 with value: 0.37577272942702517.
[I 2021-06-23 19:14:38,773] Trial 107 finished with value: 0.6872706197069718 and paramet

[I 2021-06-23 19:14:39,091] Trial 137 finished with value: 0.3808962908832274 and parameters: {'n_1': 0.30330208631972794, 'n_2': 0.0009081973570225377, 'n_3': 0.6258354646370264}. Best is trial 126 with value: 0.37187343760568087.
[I 2021-06-23 19:14:39,100] Trial 138 finished with value: 0.3743199915257916 and parameters: {'n_1': 0.33565260247832274, 'n_2': 0.0515873873098954, 'n_3': 0.594296199986649}. Best is trial 126 with value: 0.37187343760568087.
[I 2021-06-23 19:14:39,113] Trial 139 finished with value: 0.37491775999568566 and parameters: {'n_1': 0.33750611357174726, 'n_2': 0.041957681797617524, 'n_3': 0.5885182598142149}. Best is trial 126 with value: 0.37187343760568087.
[I 2021-06-23 19:14:39,122] Trial 140 finished with value: 0.38195201311240706 and parameters: {'n_1': 0.30755846721735225, 'n_2': 0.05834385713291199, 'n_3': 0.6756282754084121}. Best is trial 126 with value: 0.37187343760568087.
[I 2021-06-23 19:14:39,132] Trial 141 finished with value: 0.3744516181599644

...

[I 2021-06-23 19:14:49,572] Trial 885 finished with value: 0.4093711219835244 and parameters: {'n_1': 0.4505525382604519, 'n_2': 0.1644968388306406, 'n_3': 0.5002469639548969}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:49,590] Trial 886 finished with value: 0.3697462655900753 and parameters: {'n_1': 0.4657937469387582, 'n_2': 0.00036376711305743676, 'n_3': 0.5245629900330634}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:49,607] Trial 887 finished with value: 0.37158136657847357 and parameters: {'n_1': 0.43691263531098923, 'n_2': 0.017668055847538108, 'n_3': 0.5580652922755988}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:49,624] Trial 888 finished with value: 0.4025546777544875 and parameters: {'n_1': 0.4881436625382765, 'n_2': 0.03428023283705205, 'n_3': 0.5871920685814466}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:49,642] Trial 889 finished with value: 0.3715563952146391 and p

[I 2021-06-23 19:14:50,179] Trial 919 finished with value: 0.3742601835526943 and parameters: {'n_1': 0.4418779628919359, 'n_2': 0.017535101626658802, 'n_3': 0.572449006432792}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:50,197] Trial 920 finished with value: 0.39432321727564473 and parameters: {'n_1': 0.4744739935951734, 'n_2': 0.0634995329210108, 'n_3': 0.5531650318033768}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:50,214] Trial 921 finished with value: 0.37495464183056837 and parameters: {'n_1': 0.416848216458546, 'n_2': 0.0338509146339609, 'n_3': 0.5090459633878675}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:50,232] Trial 922 finished with value: 0.3710048245041717 and parameters: {'n_1': 0.46015276877218514, 'n_2': 0.018953096354936007, 'n_3': 0.5281124779996502}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:50,249] Trial 923 finished with value: 0.37824498463377065 and para

[I 2021-06-23 19:14:50,819] Trial 953 finished with value: 0.3701505778275203 and parameters: {'n_1': 0.49215823272323794, 'n_2': 0.0007273506565298749, 'n_3': 0.5168584887553794}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:50,837] Trial 954 finished with value: 0.3698489111212605 and parameters: {'n_1': 0.46109295417489704, 'n_2': 2.441297176654833e-05, 'n_3': 0.5414128811942347}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:50,855] Trial 955 finished with value: 0.37089407072809577 and parameters: {'n_1': 0.4445735010925578, 'n_2': 0.0008180398808267485, 'n_3': 0.5687051118695673}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:50,874] Trial 956 finished with value: 0.38117433392622424 and parameters: {'n_1': 0.5099973474066486, 'n_2': 0.036345472969252314, 'n_3': 0.5139928359478553}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:50,892] Trial 957 finished with value: 0.3734955419298935

[I 2021-06-23 19:14:51,464] Trial 987 finished with value: 0.38474822008010756 and parameters: {'n_1': 0.4723962118317615, 'n_2': 0.01872101003387765, 'n_3': 0.5795059392731476}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:51,482] Trial 988 finished with value: 0.3730223683543209 and parameters: {'n_1': 0.43408755247693975, 'n_2': 0.042583270848776826, 'n_3': 0.5006008470953593}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:51,499] Trial 989 finished with value: 0.3699840069480146 and parameters: {'n_1': 0.4516215906314653, 'n_2': 0.0006077979035086774, 'n_3': 0.5331431423939281}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:51,517] Trial 990 finished with value: 0.39078146813157666 and parameters: {'n_1': 0.48641349744546253, 'n_2': 0.0001779734890327596, 'n_3': 0.6003530930808797}. Best is trial 824 with value: 0.3696754822311988.
[I 2021-06-23 19:14:51,536] Trial 991 finished with value: 0.37734339744780915 

Number of finished trials: 1000
Best trial:
  Value: 0.3696754822311988
  Params: 
    n_1: 0.4957796898371655
    n_2: 8.992617273846838e-05
    n_3: 0.5016586246194995


In [69]:
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

  Value: 0.3696754822311988
  Params: 
    n_1: 0.4957796898371655
    n_2: 8.992617273846838e-05
    n_3: 0.5016586246194995


In [70]:
pred = (0.4957796898371655*pred_1 + 8.992617273846838e-05*pred_2 + 0.5016586246194995*pred_3)
print (np.sqrt(mean_squared_error(pred, target)))

0.3696754822311988
